# Unit-testing functions that return dictionaries

Dictionaries don't remember the order in which you inserted elements. The following code initializes `d1['h']` and `d1['e']` in the opposite order, but `d1` and `d2` print the same:

In [1]:
d1 = dict()
d1['h'] = 1
d1['e'] = 2
print('d1 =', d1)

d2 = dict()
d2['e'] = 2
d2['h'] = 1
print('d2 =', d1)

d1 = {'h': 1, 'e': 2}
d2 = {'h': 1, 'e': 2}


We can verify that the keys are in the same order by looping over them.

In [6]:
for i, k in enumerate(d1.keys()):
    print("d1's %dth key = %r" % (i, k))

for i, k in enumerate(d2.keys()):
    print("d2's %dth key = %r" % (i, k))

d1's 0th key = 'h'
d1's 1th key = 'e'
d2's 0th key = 'h'
d2's 1th key = 'e'


The code above is an example where *different* code creates dictionaries with their keys in the *same* order.

What you don't see is that the *same* code can create a dictionary with its keys in a *different* order – if you run the code one day and then the next, or on two different computers.

This poses a problem for unit testing. Here's an implementation of `histogram` from the Day 7 Reading Journal, with a unit test that worked one day. It fails today.

In [13]:
def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> histogram("hello")
    {'h': 1, 'l': 2, 'o': 1, 'e': 1}
    """
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())

**********************************************************************
File "__main__", line 3, in NoName
Failed example:
    histogram("hello")
Expected:
    {'h': 1, 'l': 2, 'o': 1, 'e': 1}
Got:
    {'h': 1, 'o': 1, 'l': 2, 'e': 1}


The problem is that yesterday printed representation includes the `'e'` key before the `'o'` key.
Today's printed representation includes these two keys in the opposite order.

doctest compares the printed representations, so yesterday's dict counts as different from today's dict – this even though Python considers the two dictionaries equal:

In [14]:
{'h': 1, 'l': 2, 'o': 1, 'e': 1} == {'h': 1, 'o': 1, 'l': 2, 'e': 1}

True

We could change the doctest to expect `{'h': 1, 'o': 1, 'l': 2, 'e': 1}` instead of `{'h': 1, 'l': 2, 'o': 1, 'e': 1}`.
This would fix the test today; it might just break again tomorrow.

Here are some more robust workarounds.

In [15]:
# Test whether Python considers the values equal, rather than whether their printed representations are the same.

def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> histogram("hello") == {'h': 1, 'e': 1, 'l': 2, 'o': 1}
    True
    """
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())

In [47]:
# The solution in the previous cell has the disadvantage that if the test *fails*, the expected and
# actual values aren't printed.

def broken_histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> broken_histogram("hello") == {'h': 1, 'l': 2, 'o': 1, 'e': 1}
    True
    """
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    # make it fail:
    del d['e']
    return d

import doctest
doctest.run_docstring_examples(broken_histogram, globals())

**********************************************************************
File "__main__", line 6, in NoName
Failed example:
    broken_histogram("hello") == {'h': 1, 'l': 2, 'o': 1, 'e': 1}
Expected:
    True
Got:
    False


Another strategy is to use the return value that to create another value that doesn't depend
on the order of keys in a dictionary. One possibility is to *sort* the keys.

In [19]:
def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> sorted(histogram("hello").items())
    [('e', 1), ('h', 1), ('l', 2), ('o', 1)]
    """
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())

`('e', 1)` is guaranteed to precede `('h', 1)` because `'e'` precedes `'h'`. ([Python documentation](https://docs.python.org/3/library/stdtypes.html#common-sequence-operations).) This is called [lexicographic order](https://en.wikipedia.org/wiki/Lexicographical_order).

You can also spot-check the returned value. This is often necessary for a larger return value, that is too large to comfortably print or include in a doc string.

In [25]:
def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> h = histogram("hello")
    >>> h['h']
    1
    >>> h['l']
    2
    >>> 'q' in h
    False
    """
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())

Finally, this issue comes about because doctest compares string representations of values instead of the values themselves. doctest is wonderful but it has limitations.

If you graduate to another test framework such as (the build in) [unittest](https://docs.python.org/3/library/unittest.html) framework or the widely-adopted [pytest](http://docs.pytest.org/en/latest/) framework, you will write functions that compare values instead of string representations:

In [48]:
def test_histogram():
    assert histogram('hello') == {'h': 1, 'e': 1, 'l': 2, 'o': 1}

test_histogram()

When a test fails, these  test frameworks report both the expected and actual values.

In the block below, where the test is run without any framework, this happens because the value is in the source code and Python prints the offendng line of source code.

A test framework prints the values even when they're computed.

In [49]:
def test_broken_histogram():
    assert broken_histogram('hello') == {'h': 1, 'e': 1, 'l': 2, 'o': 1}

test_broken_histogram()

AssertionError: 

Finally, you could use an OrderedDict instead of an (unordered) dictionary. An OrderedDict is a dict that remembers the order that its keys were inserted:

In [52]:
from collections import OrderedDict

def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> histogram("hello")
    OrderedDict([('h', 1), ('e', 1), ('l', 2), ('o', 1)])
    """
    d = OrderedDict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())

Unlike with a `dict`, the order of keys in the printed representation of the OrderedDict is *guaranteed*.

An OrderedDict has the same API as a dict (code that expects a dict will work with an OrderedDict), but the printed representation, as you can see, is different.

Note that since it has the same API, tests that use the spot-check approach instead of requiring the printed representation, don't need to be changed.

In [53]:
from collections import OrderedDict

def histogram(s):
    """Return a dict that maps each character in s to the count of its occurrences in s.
    >>> h = histogram("hello")
    >>> h['h']
    1
    >>> h['l']
    2
    >>> 'q' in h
    False
    """
    d = OrderedDict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

import doctest
doctest.run_docstring_examples(histogram, globals())